In [1]:
# importing library for data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import sql library
import sqlite3
# import text library for cleaning text data 
import re
# import  API library 
import flask
import flasgger


data  =  pd.read_csv('data/data.csv', encoding = 'latin-1')
kamusalay = pd.read_csv('data/new_kamusalay.csv', encoding='latin-1', header=None)
abusive_dict = pd.read_csv('data/abusive.csv', encoding='latin-1', header=None)

# pamggil kamus alay 
kamusalay = kamusalay.rename(columns={0: 'original', 
                                      1: 'replacement'})

abusive_dict = abusive_dict.rename(columns={0: 'abusive'})

#preprocessing

import re #  imprting regular expression library 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# fungsi
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    pattern = re.compile(r'\\x[0-9A-Fa-f]{2}')
# Use the compiled pattern to replace matches in the text
    text = pattern.sub(' ', text)
    #text = re.sub((r'\\x[0-9A-Fa-f]{2}'),' ', text) #remove emoji 
    text = re.sub('  +', ' ', text) # Remove extra spaces 
    return text
   
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

kamusalay_map = dict(zip(kamusalay['original'], kamusalay['replacement']))
def normalize_alay(text):
    return ' '.join([kamusalay_map[word] if word in kamusalay_map else word for word in text.split(' ')])


def remove_abusive(text):
    text = ' '.join(['' if word in abusive_dict.abusive.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text


def stemming(text):
    return stemmer.stem(text)


def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    text = remove_abusive(text) # 5
    return text

#applying changes
data['Tweet'] = data['Tweet'].apply(preprocess)

data.to_csv('cleaned_data.csv', index=False)